# Caribbean Natural Landcover in Floodplains

This is an indicator for the 2023 base blueprint.

Created by Amy Keister, last run by Amy Keister 15 Feb 2023. It took 4 minutes to run.

In [78]:
import os
import arcpy

In [79]:
import time
start = time.time()

In [80]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanNat\CaribbeanNatu.gdb"

In [81]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanNat\CaribbeanNaturalLandcoverInFloodplains.tif"

In [82]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [83]:
# define inputs
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"
lulc= r"F:\GIS_DATA\LanduseLandcover\Landfire\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"
FP = r"F:\GIS_DATA\WaterResources\VIPR_FEMA_FLOODPLAIN\PRVI_Floodplain_5070.shp"
flow= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NetworkNHDFlowline"

# there is a large missing catchment on the west coast of PR. I made this to fill it in
missing= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\MissingCatchPR.shp"

### Start Analysis

In [84]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [85]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\CaribbeanNat\CaribbeanNatu.gdb


### Wrangle Floodplain

In [9]:
# make a copy of the flooplain vector
arcpy.management.CopyFeatures(FP, "FP", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\FP'>

In [10]:
# add field to floodplain to prepare to convert to raster
arcpy.management.CalculateField("FP", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\FP'>

In [11]:
# convert floodplain to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("FP", "raster", "FPr", "CELL_CENTER", "NONE", Rextent, "BUILD")

### Explore adding in buffered flow lines

In [12]:
# make a copy of the flowlines in the study area
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(flow, "flow", '', None, None, None)

In [13]:
# extract the stream/river lines
# need to check with Kat to see which lines she uses
arcpy.analysis.Select("flow", "RS", "ftype IN (460, 558)")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\RS'>

In [14]:
# buffer
# need to check with Kat to see how large of a buffer she uses
#arcpy.analysis.Buffer("RS", "RS30mBuff", "30 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [15]:
# add field to prepare to convert to raster
arcpy.management.CalculateField("RS", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\RS'>

In [16]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("RS", "raster", "RSr", 30)

In [17]:
# use and expand to "buffer" the streams by 1 cell, this should create total stream width of ~90 meters
out_raster = arcpy.sa.Expand("RSr", 1, [1], ""); out_raster.save("RSrExpand1")

### Combine floodplains and buffered flow lines

In [18]:
# mosaic the floodplain and buffered rasters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.MosaicToNewRaster("FPr;RSrExpand1", OutWorkspace, "FPflow", None, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

### Clip to floodplain and buffered flow lines

In [19]:
# Limit landcover to the floodplain
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con("FPflow", lulc, '',"Value = 1"); out_raster.save("lulcFP")

In [20]:
# reclassify if using NLCD
#out_raster = arcpy.sa.Reclassify("nlcdFP", "VALUE", "11 1;21 0;22 0;23 0;24 0;31 1;41 1;42 1;43 1;52 1;71 1;81 0;82 0;90 1;95 1", "DATA"); out_raster.save("lulcFPnat")

In [21]:
# Reclassify Landfire similarly to how we used NLCD to determine "natural"

# check with Kat and indicator team to see if they like this way to divide natural from not natural

out_raster = arcpy.sa.Con("lulcFP", 1, 0, "Value NOT IN (7295, 7296, 7297, 7298, 7299, 7300, 7754, 7755, 7838)"); out_raster.save("lulcFPnat")

### zonal statistics

In [22]:
# add in missing catchment on west side of Puerto Rico
# I had created this catchment to fill in the gap. It is a weird error with sevearl narrow "catchments" 
# circling a donut hole

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Union([catch, missing], "catch", "ALL", None, "NO_GAPS")

In [23]:
# zonal statistics. calculate mean natural cover in floodplain per catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ZonalStatistics("catch", "GRIDCODE", "lulcFPnat", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchNatFP")

In [24]:
# times
out_raster = arcpy.sa.Times("ZStatNHDCatchNatFP", 100); out_raster.save("ZStatNHDCatchNatFPx100")

In [25]:
# Limit to the floodplain. Have to do this again since my zonal stat was by catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con("FPflow", "ZStatNHDCatchNatFPx100", '',"Value = 1"); out_raster.save("ind")

In [26]:
# reclassify

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("ind", "VALUE", "NODATA 0;0 60 1;60 70 2;70 80 3;80 90 4;90 100 5", "DATA"); out_raster.save("prelim")

### Prep data to make a zero class
zeros will be areas where there are NHD data, and NoData will be areas where there is not NHD data

In [27]:
# add field to the catchment layer to prepare to convert to raster
arcpy.management.CalculateField("catch", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\catch'>

In [28]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("catch", "raster", "catch1", 30)

In [29]:
# clip back down to NHD catchment area
out_raster= arcpy.sa.Times("prelim", "catch1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [30]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 5:
        return '5 = >90% natural landcover within the estimated floodplain, by catchment'
    elif v == 4:
        return '4 = >80-90%'
    elif v == 3:
        return '3 = >70-80'
    elif v == 2:
        return '2 = >60-70%'
    elif v == 1:
        return '1 = ≤60% natural landcover within the estimated floodplain, by catchment'   
    elif v == 0:
        return '0 = Not identified as a floodplain'   
"""

In [31]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNat\\CaribbeanNatu.gdb\\indicator'>

In [86]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 9
	if Value == 4:
		return 61
	if Value == 3:
		return 112
	if Value == 2:
		return 152
	if Value == 1:
		return 182
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 13
	if Value == 4:
		return 77
	if Value == 3:
		return 144
	if Value == 2:
		return 199
	if Value == 1:
		return 239
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 94
	if Value == 4:
		return 138
	if Value == 3:
		return 161
	if Value == 2:
		return 173
	if Value == 1:
		return 182
	else:
		return 255
"""

In [87]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [88]:
# clip to extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("indicator1")

In [89]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("indicator1", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [36]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

300.0489420890808
